## Overview
For the 20-21 Resident Assistant process, this code was used to clean data exported from the Microsoft forms containing the RA applications, returner intent to return, and residential data. The code merges information together to form one spreadsheet for the pool.

In [1]:
import pandas as pd

## New Applicants

In [ ]:
new_apps = pd.read_excel('Resident Advisor Application.xlsx')

In [ ]:
#Change column names
new_column_name_changes = {
   
    'Name2':'Name',
    
    'Do you plan to study abroad during the 2020-2021 academic year? Plans to study abroad do NOT automatically disqualify you from being an RA.':'Study Abroad Plans',
    "Please tell us about any leadership or extracurricular involvements that you've had during your time at UVM thus far. What are some things you are also planning to become involved in the next acad...":'Extracurriculars',
    'Please indicate which learning community you are a part of for the 2020-2021 academic year.':'Current LC',
       
    'Which Learning Community is your FIRST preference':'LC Preference - First',
    'Which Learning Community is your SECOND preference':'LC Preference - Second',
    'Which Learning Community is your THIRD preference':'LC Preference - Third',
    'In the event, you are selected as an RA but we are not able to place you within your top three preferences, please indicate any other learning communities you would like to be considered for. PLEA...':'Other LCs of Interest',
    
    'Are you interested in potentially being in the Wellness Environment as an RA, please indicate whether you have successfully completed the required COMU001: Healthy Brains, Healthy Bodies (HBHB) Co...':'Qualify for WE',
    'Are you a current HONORS COLLEGE student (regardless of whether you live in the Honors College community or not)':'Honors',
    
    'Do you plan to apply for a Community Student Leader (CSL) position, regardless of whether you are an RA or not?':'Intent to Apply for CSL',
    ' If yes, please indicate where. If not, please respond “N/A”':'Intent to Apply to CSL - Where',
    
    'Please tell us about a time in which you had to engage with someone with different identities from you (i.e., race, sex, gender, sexual identity, nationality, class, ability, religion, veteran sta...':'Prompt 1 Response',
    'What are some of the core values and principles that guide your actions? How do you see these values influencing your performance as an RA? (300 word limit max)':'Prompt 2 Response',
    'Perceiving and addressing the emotions of others is an important part of building a community. How do you respond to the emotions of others to build and strengthen relationships? (300-word limit max)':'Prompt 3 Response',
    'Tell us how you go about meeting new people? What do you find rewarding about building new relationships? What are some challenges you face in meeting new people and building relationships? How ha...':'Prompt 4 Response',
    'Tell us about a time you were involved in a conflict. Walk us through how the conflict was addressed. What happened at the end of the conflict for everyone involved? (300-word limit max)':'Prompt 5 Response',
    
    'Interviews Reccommendations':'Interview Recommendations'
}

new_apps=new_apps.rename(columns = new_column_name_changes)

In [ ]:
#Select relevant columns
relevant_columns = [
    'ID', 'Status', 'Name',
    'Email', 'Net ID', 'Cell Phone Number',
    'Number of Semesters Completed at UVM', 'Anticipated Graduation Year',
    'Academic College', 'Study Abroad Plans',
    'Extracurriculars', 'Current LC', 'LC Preference - First',
    'LC Preference - Second', 'LC Preference - Third',
    'Other LCs of Interest', 'Qualify for WE', 'Honors',
    'Intent to Apply for CSL', 'Intent to Apply to CSL - Where',
    'Prompt 1 Response', 'Prompt 2 Response', 'Prompt 3 Response',
    'Prompt 4 Response', 'Prompt 5 Response',
    'Application Review 1', 'Application Score 1',
    'Application Rationale 1', 'Application Reviewer 2',
    'Application Score 2', 'Application Rationale 2',
    'Application Total Score', 'Interviewer 1', 'Interview Score 1',
    'Interview Recommendation 1', 'Interview Rational 1', 'Interviewer 2',
    'Interview Score 2', 'Interview Recommendation 2',
    'Interview Rational 2', 'Interview Recommendations',
    'Hired for MidYear (Y/N)', 'MidYear Community Placement',
    'MidYear Room Placement', 'Candidate Acceptance'
]

new_apps_revised=new_apps[relevant_columns].copy()

In [ ]:
#Calculate avg and total scores and standardizes interview rcommendations

new_apps_revised['Avg. Application Score']=(new_apps_revised['Application Score 1']+new_apps_revised['Application Score 2'])/2
new_apps_revised['Avg. Interview Score']=(new_apps_revised['Interview Score 1']+new_apps_revised['Interview Score 2'])/2
new_apps_revised['Total Interview Score']=(new_apps_revised['Interview Score 1']+new_apps_revised['Interview Score 2'])

standardize_interview_recommendations = {  
    'Recommend with RESERVATIONS & Recommend for Hire':'Recommend for Hire & Recommend with RESERVATIONS',
    ' & ':'No Interview Conducted',
    'Do NOT Recommend for Hire & Recommend with RESERVATIONS':'Recommend with RESERVATIONS & Do NOT Recommend for Hire',
    'Do NOT Recommend for Hire':'Recommend for Hire & Do NOT Recommend for Hire'
}
new_apps_revised['Interview Recommendations'].replace(standardize_interview_recommendations,inplace=True)

standardize_honors = {
    'I AM NOT a honors college student':'No', 'I AM a honors college student':'Yes'
}
new_apps_revised['Honors'].replace(standardize_honors,inplace=True)

standardize_WE = {
    'I haven NOT SUCCESSFULLY completed the required HBHB course':'Yes',
    'I have SUCCESSFULLY completed the Required HBHB coursee':'No'
}
new_apps_revised['Qualify for WE'].replace(standardize_WE,inplace=True)

In [ ]:
#Merge in residential info
rd_roster = pd.read_csv('SP21-AllResidence.csv')
rd_roster.drop_duplicates(subset=['NetID'],keep='first')
rd_roster['Current Building']=rd_roster['Room Space Description'].str.split('-',expand=True).rename(columns = {0:'Current Building'})['Current Building']
new_apps_merged = pd.merge(left=new_apps_revised,right=rd_roster[['Student Number','LC Placement','Gender Identity','Major','NetID','Current Building']],left_on='Net ID',right_on='NetID',how='left')

In [ ]:
#Sets applicant group column
new_apps_merged['Applicant Group'] = new_apps_merged['MidYear Room Placement'].apply(lambda x: 'New' if pd.isnull(x) else 'Spr 21 Hire')

#Creates column for LCs of Interest
new_apps_merged['LCs of Interest']= new_apps_merged['LC Preference - First'] + ', ' + new_apps_merged['LC Preference - Second'] + ', ' + new_apps_merged['LC Preference - Third']

In [ ]:
new_apps_merged = new_apps_merged[[
    'ID','Student Number', 'Name', 'Email', 'Net ID', 'Applicant Group',
    'Cell Phone Number', 'Gender Identity', 'Major', 'Number of Semesters Completed at UVM',
    'Anticipated Graduation Year', 'Academic College', 'Study Abroad Plans',
    'Extracurriculars','Current Building', 'LC Placement', 'LC Preference - First',
    'LC Preference - Second', 'LC Preference - Third', 'LCs of Interest',
    'Other LCs of Interest', 'Qualify for WE', 'Honors',
    'Intent to Apply for CSL', 'Intent to Apply to CSL - Where',
    'Prompt 1 Response', 'Prompt 2 Response', 'Prompt 3 Response',
    'Prompt 4 Response', 'Prompt 5 Response', 'Application Review 1',
    'Application Score 1', 'Application Rationale 1',
    'Application Reviewer 2', 'Application Score 2',
    'Application Rationale 2', 'Application Total Score',  'Avg. Application Score', 'Interviewer 1',
    'Interview Score 1', 'Interview Recommendation 1',
    'Interview Rational 1', 'Interviewer 2', 'Interview Score 2',
    'Interview Recommendation 2', 'Interview Rational 2','Total Interview Score',    
    'Avg. Interview Score','Interview Recommendations', 'Hired for MidYear (Y/N)',
    'MidYear Community Placement', 'MidYear Room Placement',
    'Candidate Acceptance',
]]

new_apps_merged

## Returners

In [ ]:
#Bring in dataset
returner_intent=pd.read_excel('RA Intent to Return Academic Year 2021-22(1-125).xlsx')

In [ ]:
returner_intent.columns

In [ ]:
returner_intent.head(3)

In [ ]:
returner_intent['Name']=returner_intent['First Name']+' '+returner_intent['Last name']
returner_intent['Applicant Group']='Returner'

In [ ]:
#Remove duplicates. Take the most recent submission
returner_intent.drop_duplicates(subset=['Email'],keep='last',inplace=True)

#Rename columns as needed
returner_column_name_change = {
'Will you return to campus and your RA role for 2021-22?':'Intent to Return',
'If no, please tell us why you will not be returning':'Intent to Return, No - Why',
"If you intend to return: Please select three Learning Communities where you are interested in working for the '21-'22 year. If you do not select three you will be placed based on the needs of the ...":'LCs of Interest',
'Have you completed the HBHB (WE Course)?':'Qualify for WE',
'Are you currently part of the Honors College?':'Honors'
}

returner_intent=returner_intent.rename(columns = returner_column_name_change)

#Checking response of mid-year hres
spr_21_hires_df=new_apps_merged[new_apps_merged['Applicant Group']=='Spr 21 Hire'][['Email']].copy()
spr_21_hires_df= pd.merge(left=spr_21_hires_df,right=returner_intent[['Email','Intent to Return']],left_on='Email',right_on='Email',how='left')

#Filters for those hoping to return
returner_intent=returner_intent[returner_intent['Intent to Return']=='Yes']

#Filter for those hired in Spring 21
spr_21_hires=new_apps_merged[new_apps_merged['Applicant Group']=='Spr 21 Hire']['Email']
returner_intent=returner_intent[~returner_intent['Email'].isin(spr_21_hires)]

#Make df for relevant columns
relevant_columns = [ 'Name','Email',
'LCs of Interest',
'Qualify for WE','Honors','Applicant Group']
returner_intent_revised=returner_intent[relevant_columns].copy()

#Creates netid column
returner_intent_revised['Net ID']=returner_intent_revised['Email'].str[:-8]

#Merge info from residential Roster
returner_intent_merged = pd.merge(left=returner_intent_revised,right=rd_roster[['Student Number','LC Placement','Gender Identity','Major','NetID','Current Building']],left_on='Net ID',right_on='NetID',how='left').drop(columns='NetID')




In [ ]:
spr_21_hires_df

## Creating Union

In [ ]:
#Creates data union for new and returner datasets
data = [new_apps_merged,returner_intent_merged]
complete_df = pd.concat(data,sort=True)

#Merging in MidYear Hire Intent
complete_df= pd.merge(left=spr_21_hires_df,right=complete_df,left_on='Email',right_on='Email',how='right').rename(columns= {'Intent to Return':'MidYear Hire Intent to Return'})

complete_df = complete_df[[
    'ID','Student Number', 'Name', 'Email', 'Net ID', 'Applicant Group',
    'Cell Phone Number', 'Gender Identity', 'Major', 'Number of Semesters Completed at UVM',
    'Anticipated Graduation Year', 'Academic College', 'Study Abroad Plans',
    'Extracurriculars', 'Current Building','LC Placement', 'LC Preference - First',
    'LC Preference - Second', 'LC Preference - Third',
    'Other LCs of Interest', 'LCs of Interest', 'Qualify for WE', 'Honors',
    'Intent to Apply for CSL', 'Intent to Apply to CSL - Where',
    'Prompt 1 Response', 'Prompt 2 Response', 'Prompt 3 Response',
    'Prompt 4 Response', 'Prompt 5 Response', 'Application Review 1',
    'Application Score 1', 'Application Rationale 1',
    'Application Reviewer 2', 'Application Score 2',
    'Application Rationale 2', 'Application Total Score','Avg. Application Score', 'Interviewer 1',
    'Interview Score 1', 'Interview Recommendation 1',
    'Interview Rational 1', 'Interviewer 2', 'Interview Score 2',
    'Interview Recommendation 2', 'Interview Rational 2','Avg. Interview Score', 'Total Interview Score' , 
    'Interview Recommendations', 'Hired for MidYear (Y/N)',
    'MidYear Community Placement', 'MidYear Room Placement',
    'Candidate Acceptance', 'MidYear Hire Intent to Return'
     
]]

complete_df[['Hired Status','Placement']] =''

complete_df['LC Placement']=complete_df['LC Placement'].apply(lambda x: 'Not A Current Resident' if pd.isnull(x) else x)



complete_df.head(5)

In [ ]:
complete_df['Name'].split()

In [ ]:
complete_df

## Export

In [ ]:
#This section is completely commented out to avoid creating excel export.

# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('Complete RA Pool.xlsx', engine='xlsxwriter')

# Position the dataframes in the worksheet.
complete_df.to_excel(writer, sheet_name='Sheet1')  # Default position, cell A1.

# Close the Pandas Excel writer and output the Excel file.
writer.save()